In [481]:
import selenium 
import pandas as pd
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import os
import urllib
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait       
from selenium.webdriver.common.by import By       
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.keys import Keys

In [298]:
drivers= webdriver.Chrome(r"C://chromedriver.exe")

# Q1.Write a python program which searches all the product under a particular product from
www.amazon.in. The product name to be searched will be taken as input from user. For e.g. If
user input is ‘guitar’. Then search for guitars.

In [169]:
drivers.get("https://www.amazon.in/")

In [170]:
#Input Search
search_item=drivers.find_element_by_xpath('//input[@id="twotabsearchtextbox"]')
productname= input("Enter the product name : ")
search_item.send_keys(productname)

#Click Search Button
search_btn=drivers.find_element_by_xpath('//input[@id="nav-search-submit-button"]')
search_btn.click()

Enter the product name : guitars


# Q2.In the above question, now scrape the following details of each product listed in first 3 pages
of your search results and save it in a dataframe and csv. In case if any product vertical has
less than 3 pages in search results then scrape all the products available under that product
vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of
Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details"
and “Product URL”. In case, if any of the details are missing for any of the product then
replace it by “-“.

In [171]:
Brand_Name = []
Name_of_the_Product = []
Rating = []
No_of_Ratings = []
Price = []
Return_Exchange = []
Expected_Delivery = []
Availability = []
Other_Details = []
URL = []

In [173]:

j=1
while j<=3:    
    product_url=drivers.find_elements_by_xpath('//a[@class="a-link-normal s-no-outline"]')
    for i in product_url:
        URL.append(i.get_attribute('href'))    
    try:
        drivers.find_elements_by_xpath('//div[@class="a-text-center"]/ul/li')[-1].click() 
        time.sleep(2)
    except:
        print("End")    
    j+=1

In [178]:
for url in URL:
    drivers.get(url)
#Brand_Name        
    try:
        brand=drivers.find_element_by_xpath('//table[@id="productDetails_techSpec_section_1"]/tb')[1]
        Brand_Name.append(brand.text)
        
    except NoSuchElementException as e:
        Brand_Name.append("-")
#Return_Exchange                                            
    try:
        Return=drivers.find_element_by_xpath('//div[@id="RETURNS_POLICY"]')
        Return_Exchange.append(Return.text)
        
    except NoSuchElementException as e:
        Return_Exchange.append("-")
#Expected_Delivery                                            
    try:
        delivery=drivers.find_element_by_xpath('//span[@id="upsell-message"]/b')
        Expected_Delivery.append(delivery.text)  
        
    except NoSuchElementException as e:
        Expected_Delivery.append("-")
#Availability
    try:
        availb=drivers.find_element_by_xpath('//span[@class="a-size-medium a-color-success"]')
        Availability.append(availb.text)  
        
    except NoSuchElementException as e:
        Availability.append("-")
#Other_Details
    try:
        otherd=drivers.find_element_by_xpath('//div[@id="productDescription"]')
        Other_Details.append(otherd.text)
        
    except NoSuchElementException as e:
        Other_Details.append("-")
#price
    try:
        price=drivers.find_element_by_xpath('//span[@id="priceblock_ourprice"]')
        Price.append(price.text.strip(" ").replace('\n',''))
        
    except NoSuchElementException as e:
        Price.append("-")
#Name of the product
    try:
        Name_tags=drivers.find_element_by_xpath('//span[@id="productTitle"]')
        Name_of_the_Product.append(Name_tags.text.strip(" ").replace('\n',''))
        
    except NoSuchElementException as e:
        Name_of_the_Product.append("-")
        
#Rating
#No_of_Ratings                                            
    try:
        rate=drivers.find_element_by_xpath('//span[@id="acrCustomerReviewText"]')
        No_of_Ratings.append(rate.text)        # No of Ratings                            
        WebDriverWait(drivers, 20).until(EC.element_to_be_clickable((By.XPATH, '//span[@id="acrCustomerReviewText"]'))).click()
        rating=drivers.find_element_by_xpath('//span[@class="a-size-medium a-color-base"]')
        Rating.append(rating.text)     # Collecting Ratings
        
    except NoSuchElementException as e:
        Rating.append("-") 
        No_of_Ratings.append("-")

In [179]:
data= list(zip(Name_of_the_Product,Rating,No_of_Ratings,Price,Return_Exchange,Expected_Delivery,Availability,Other_Details,URL,Brand_Name))

In [180]:
df= pd.DataFrame(data,columns=["Name_of_the_Product","Rating","No_of_Ratings","Price","Return_Exchange","Expected_Delivery","Availability","Other_Details","Product_URL","Brand_Name"])

In [182]:
df.head(50)

,Name_of_the_Product,Rating,No_of_Ratings,Price,Return_Exchange,Expected_Delivery,Availability,Other_Details,Product_URL,Brand_Name
0,Giuson Venus Graphics Series Rosewood Fretboar...,-,-,"₹ 3,790.00",7 Days Replacement,-,In stock.,Giuson Acoustic guitars are simply outstanding...,https://www.amazon.in/gp/slredirect/picassoRed...,-
1,GUITAR BRO - COMBO (Blue Acoustic Guitar for B...,-,18 ratings,"₹ 6,499.00",7 Days Replacement,-,In stock.,-,https://www.amazon.in/gp/slredirect/picassoRed...,-
2,Giuson Venus Graphics Series Rosewood Fretboar...,-,-,"₹ 3,790.00",7 Days Replacement,-,In stock.,Giuson Acoustic guitars are simply outstanding...,https://www.amazon.in/gp/slredirect/picassoRed...,-
3,GUITAR BRO - COMBO (Blue Acoustic Guitar for B...,4.1 out of 5,18 ratings,"₹ 6,499.00",7 Days Replacement,-,In stock.,-,https://www.amazon.in/gp/slredirect/picassoRed...,-
4,Giuson Venus Graphics Series Rosewood Fretboar...,3.1 out of 5,-,"₹ 3,790.00",7 Days Replacement,-,In stock.,Giuson Acoustic guitars are simply outstanding...,https://www.amazon.in/Juarez-Acoustic-Cutaway-...,-
5,GUITAR BRO - COMBO (Blue Acoustic Guitar for B...,3.3 out of 5,18 ratings,"₹ 6,499.00",7 Days Replacement,-,In stock.,-,https://www.amazon.in/JUAREZ-JRZ38C-Acoustic-S...,-
6,Blueberry 38C IN TrussRod Acoustic Guitar Kit ...,3.9 out of 5,4 ratings,"₹ 2,895.00",7 Days Replacement,-,In stock.,Blueberry Deigned 38c 38 inch Cutway guitar in...,https://www.amazon.in/Intern-INT-38C-Acoustic-...,-
7,Blueberry 38 Inch Acoustic Guitar Kit Inbuilt ...,4 out of 5,45 ratings,"₹ 2,999.00",7 Days Replacement,-,In stock.,-,https://www.amazon.in/Juarez-Acoustic-Cutaway-...,-
8,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",3.9 out of 5,"8,915 ratings","₹ 2,499.00",7 Days Replacement,"Monday, April 26",In stock.,If you are a music lover and a fan of acoustic...,https://www.amazon.in/Intern-INT-38C-Acoustic-...,-
9,"Juarez JRZ38C Acoustic Guitar, 38 Inch Cutaway...",3.9 out of 5,"3,059 ratings","₹ 2,499.00",7 Days Replacement,"Monday, April 26",In stock.,If you are a music lover and a fan of acoustic...,https://www.amazon.in/RockJam-RJEG02-SK-RD-Ele...,-


# Q3.Write a python program to access the search bar and search button on images.google.com andscrape 100 images 
each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [450]:
drivers= webdriver.Chrome(r"C://chromedriver.exe")
drivers.get("https://images.google.com/?gws_rd=ssl")

In [451]:
#Input Search
search_item=drivers.find_element_by_xpath('//input[@class="gLFyf gsfi"]')
productname= input("Enter the keyword : ")
search_item.send_keys(productname)

#Click Search Button
search_btn=drivers.find_element_by_xpath('//span[@class="z1asCe MZy1Rb"]')
search_btn.click()

Enter the keyword : fruits


In [470]:
path = r"C://chromedriver.exe"

url_prefix = "https://www.google.com.sg/search?q="
url_postfix = "&source=lnms&tbm=isch&sa=X&ei=0eZEVbj3IJG5uATalICQAQ&ved=0CAcQ_AUoAQ&biw=939&bih=591"

save_folder = 'Train Images'

def main():
    if not os.path.exists(save_folder):
        os.mkdir(save_folder)
    download_images()
    
def download_images():
    topic = input("What do you want to search for? ")
    n_images = int(input('How many images do you want? '))
    
    search_url = url_prefix+topic+url_postfix
    #print(search_url)
    
    path = r"C://chromedriver.exe"
    
    driver = webdriver.Chrome(path)
    driver.get(search_url)
    
    value = 0
    for i in range(8):
        driver.execute_script("scrollBy("+ str(value) +",+1000);")
        value += 1000
        time.sleep(2)
    
    elem1 = driver.find_element_by_id('islmp')
    sub = elem1.find_elements_by_tag_name('img')
    
    for j,i in enumerate(sub):
        if j <= n_images:
            src = i.get_attribute('src')                         
            try:
                if src != None:
                    src  = str(src)
                    
                    urllib.request.urlretrieve(src, os.path.join(save_folder, topic+str(j)+'.jpg'))
                    
                else:
                    raise TypeError
            except Exception as e:              #catches type error along with other errors
                print(f'fail with error {e}')
    
    driver.close()
    
if __name__ == "__main__":
    main()

What do you want to search for? fruits
How many images do you want? 100


# Q4.Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) 
on www.flipkart.com and scrape following details for all the search results displayed on 1st page.
Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”,
“Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display
Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”.
Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe
and CSV.

In [204]:
drivers= webdriver.Chrome(r"C://chromedriver.exe")
drivers.get("https://www.flipkart.com/")

In [187]:
search_btn=drivers.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]')
search_btn.click()

In [205]:
#Input Search
search_item=drivers.find_element_by_xpath('//input[@class="_3704LK"]')
productname= input("Enter the mobile brand name : ")
search_item.send_keys(productname + " mobiles")

#Click Search Button
search_btn=drivers.find_element_by_xpath('//button[@class="L0Z3Pu"]')
search_btn.click()

Enter the mobile brand name : samsung


In [206]:
Brand_Name =[]
Smartphone_name = []
Colour = []
RAM = []
Storage = []
Primary_Camera = []
Secondary_Camera = []
Display_Size = []
Display_Resolution = []
Processor = []
Processor_Cores = []
Battery_Capacity = []
Price = []
Product_URL = []


In [208]:
#
product_url=drivers.find_elements_by_xpath('//a[@class="_1fQZEK"]')
for i in product_url:
        Product_URL.append(i.get_attribute('href'))
#        
for url in Product_URL:
    drivers.get(url)
#Brand_Name        
    try:
        brand=drivers.find_element_by_xpath('//span[@class="B_NuCI"]')
        Brand_Name.append(brand.text.strip(" ").replace('\n',''))
        
    except NoSuchElementException as e:
        Brand_Name.append("-")
#Clicking on Read More
        
    try:
        WebDriverWait(drivers, 20).until(EC.element_to_be_clickable((By.XPATH, '//button[@class="_2KpZ6l _1FH0tX"]'))).click()
    except NoSuchElementException as e:
        print('Not Clickable')    
#Smartphone_name                                            
    try:
        sname=drivers.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[1]/table/tbody/tr[3]/td[2]/ul/li')
        Smartphone_name.append(sname.text)
        
    except NoSuchElementException as e:
        Smartphone_name.append("-")
#Colour                                            
    try:
        colour=drivers.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[1]/table/tbody/tr[4]/td[2]/ul/li')
        Colour.append(colour.text)  
        
    except NoSuchElementException as e:
        Colour.append("-")
#RAM
    try:
        ram=drivers.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[4]/table/tbody/tr[2]/td[2]/ul/li')
        RAM.append(ram.text)
        
    except NoSuchElementException as e:
        RAM.append("-")
#Storage
    try:
        rom=drivers.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[4]/table/tbody/tr[1]/td[2]/ul/li')
        Storage.append(rom.text)
        
    except NoSuchElementException as e:
        Storage.append("-")
#price
    try:
        price=drivers.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')
        Price.append(price.text.strip(" ").replace('\n',''))
        
    except NoSuchElementException as e:
        Price.append("-")
#Primary_Camera
    try:
        pcam=drivers.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[5]/table/tbody/tr[2]/td[2]/ul/li')
        Primary_Camera.append(pcam.text.strip(" ").replace('\n',''))
        
    except NoSuchElementException as e:
        Primary_Camera.append("-")
#Secondary_Camera
    try:
        scam=drivers.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[5]/table/tbody/tr[5]/td[2]/ul/li')
        Secondary_Camera.append(scam.text.strip(" "))
        
    except NoSuchElementException as e:
        Secondary_Camera.append("-")
#Display_Size
    try:
        display=drivers.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[2]/table/tbody/tr[1]/td[2]/ul/li')
        Display_Size.append(display.text.strip(" ").replace('\n',''))
        
    except NoSuchElementException as e:
        Display_Size.append("-")
#Display_Resolution
    try:
        displayres=drivers.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[2]/table/tbody/tr[3]/td[2]/ul/li')
        Display_Resolution.append(displayres.text.strip(" ").replace('\n',''))
        
    except NoSuchElementException as e:
        Display_Resolution.append("-")
#Processor
    try:
        processor=drivers.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[3]/table/tbody/tr[2]/td[2]/ul/li')
        Processor.append(processor.text.strip(" ").replace('\n',''))
        
    except NoSuchElementException as e:
        Processor.append("-")
#Processor_Cores
    try:
        cores=drivers.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[3]/table/tbody/tr[3]/td[2]/ul/li')
        Processor_Cores.append(cores.text.strip(" ").replace('\n',''))
        
    except NoSuchElementException as e:
        Processor_Cores.append("-")
#Battery_Capacity
    try:
        battery=drivers.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[10]/table/tbody/tr/td[2]/ul/li')
        Battery_Capacity.append(battery.text.strip(" ").replace('\n',''))
        
    except NoSuchElementException as e:
        Battery_Capacity.append("-")                                             
                                             

In [209]:
data= list(zip(Brand_Name,Smartphone_name,Price,Colour,RAM,Storage,Primary_Camera,Secondary_Camera,Display_Size,Display_Resolution,Processor,Processor_Cores,Battery_Capacity,Product_URL))

In [210]:
df= pd.DataFrame(data,columns=["Brand_Name","Smartphone_name","Price","Colour","RAM","Storage","Primary_Camera","Secondary_Camera","Display_Size","Display_Resolution","Processor","Processor_Cores","Battery_Capacity","Product_URL"])

In [211]:
df

,Brand_Name,Smartphone_name,Price,Colour,RAM,Storage,Primary_Camera,Secondary_Camera,Display_Size,Display_Resolution,Processor,Processor_Cores,Battery_Capacity,Product_URL
0,"SAMSUNG Galaxy F12 (Sky Blue, 64 GB) (4 GB RAM)",Galaxy F12,"₹10,999",Sky Blue,4 GB,64 GB,48MP + 5MP + 2MP + 2MP,8MP Front Camera,16.55 cm (6.515 inch),HD+,Exynos 850,Octa Core,6000 mAh,https://www.flipkart.com/samsung-galaxy-f12-sk...
1,"SAMSUNG Galaxy F12 (Sea Green, 64 GB) (4 GB RAM)",Galaxy F12,"₹10,999",Sea Green,4 GB,64 GB,48MP + 5MP + 2MP + 2MP,8MP Front Camera,16.55 cm (6.515 inch),HD+,Exynos 850,Octa Core,6000 mAh,https://www.flipkart.com/samsung-galaxy-f12-se...
2,"SAMSUNG Galaxy F12 (Celestial Black, 64 GB) (...",Galaxy F12,"₹10,999",Celestial Black,4 GB,64 GB,48MP + 5MP + 2MP + 2MP,8MP Front Camera,16.55 cm (6.515 inch),HD+,Exynos 850,Octa Core,6000 mAh,https://www.flipkart.com/samsung-galaxy-f12-ce...
3,"SAMSUNG Galaxy M31 (Ocean Blue, 64 GB) (6 GB ...",-,"₹15,632",-,-,-,-,-,-,-,-,-,-,https://www.flipkart.com/samsung-galaxy-m31-oc...
4,"SAMSUNG Galaxy F12 (Celestial Black, 128 GB) ...",Galaxy F12,"₹11,999",Celestial Black,4 GB,128 GB,48MP + 5MP + 2MP + 2MP,8MP Front Camera,16.55 cm (6.515 inch),HD+,Exynos 850,Octa Core,6000 mAh,https://www.flipkart.com/samsung-galaxy-f12-ce...
5,"SAMSUNG Galaxy F12 (Sea Green, 128 GB) (4 GB ...",Galaxy F12,"₹11,999",Sea Green,4 GB,128 GB,48MP + 5MP + 2MP + 2MP,8MP Front Camera,16.55 cm (6.515 inch),HD+,Exynos 850,Octa Core,6000 mAh,https://www.flipkart.com/samsung-galaxy-f12-se...
6,"SAMSUNG Galaxy F41 (Fusion Green, 64 GB) (6 G...",Galaxy F41,"₹14,499",Fusion Green,6 GB,64 GB,64MP + 8MP + 5MP,32MP Front Camera,16.26 cm (6.4 inch),Full HD+,Exynos 9611,Octa Core,75.1 mm,https://www.flipkart.com/samsung-galaxy-f41-fu...
7,"SAMSUNG Galaxy F12 (Sky Blue, 128 GB) (4 GB RAM)",Galaxy F12,"₹11,999",Sky Blue,4 GB,128 GB,48MP + 5MP + 2MP + 2MP,8MP Front Camera,16.55 cm (6.515 inch),HD+,Exynos 850,Octa Core,6000 mAh,https://www.flipkart.com/samsung-galaxy-f12-sk...
8,"SAMSUNG Galaxy F41 (Fusion Black, 64 GB) (6 G...",Galaxy F41,"₹14,499",Fusion Black,6 GB,64 GB,64MP + 8MP + 5MP,32MP Front Camera,16.26 cm (6.4 inch),Full HD+,Exynos 9611,Octa Core,75.1 mm,https://www.flipkart.com/samsung-galaxy-f41-fu...
9,"SAMSUNG Galaxy F41 (Fusion Blue, 64 GB) (6 GB...",Galaxy F41,"₹14,499",Fusion Blue,6 GB,64 GB,64MP + 8MP + 5MP,32MP Front Camera,16.26 cm (6.4 inch),Full HD+,Exynos 9611,Octa Core,75.1 mm,https://www.flipkart.com/samsung-galaxy-f41-fu...


# Q5.Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [509]:
drivers= webdriver.Chrome(r"C://chromedriver.exe")
drivers.get("https://www.google.co.in/")

In [510]:
#Input Search
search_item=drivers.find_element_by_xpath('//input[@class="gLFyf gsfi"]')
city_name= input("Enter the City Name you want to know the geospatial coordinates: ")
search_item.send_keys("latitude and longitude coordinates of " + city_name )

#Click Search Button
search_btn=drivers.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[3]/center/input[1]')
search_btn.click()

#WebDriverWait(drivers, 20).until(EC.element_to_be_clickable((By.XPATH, '//input[@class="gNO89b"][1]'))).send_keys(Keys.ENTER)

try:
    cord=drivers.find_element_by_xpath('//div[@class="Z0LcW XcVN5d"]')
    print('Geospatial coordinates of '+ city_name +' '+ cord.text)
        
except NoSuchElementException as e:
    print('City not found')

Enter the City Name you want to know the geospatial coordinates: Pune
Geospatial coordinates of Pune 18.5204° N, 73.8567° E


# Q6.Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [216]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup

In [215]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36 RuxitSynthetic/1.0 v9845724245 t38550 ath9b965f92 altpub cvcv=2'}

In [217]:
url = 'https://trak.in/india-startup-funding-investment-2015/'

In [218]:
response = requests.get(url, headers = headers)

In [219]:
response.status_code

200

In [221]:
soup = BeautifulSoup(response.content, 'html.parser')

In [260]:
head_table = soup.findAll('h2',{'class':'tablepress-table-name tablepress-table-name-id-48'})

In [261]:
head_table

[<h2 class="tablepress-table-name tablepress-table-name-id-48">July, 2020</h2>]

In [228]:
sept_table = soup.findAll('table',{'id':'tablepress-50'})

In [231]:
sept_table = sept_table[0]

In [232]:
type(sept_table)

bs4.element.Tag

In [511]:
#for row in sept_table.find_all('tr'):
    #for cell in row.find_all('td'):
       #print(cell.text)

In [262]:
#options = webdriver.ChromeOptions()
#options.add_argument('--headless')
#options.add_argument('--no-sandbox')
#options.add_argument('--disable-dev-shm-usage')

In [315]:
site = 'https://trak.in/' 

In [334]:
drivers = webdriver.Chrome(r"C://chromedriver.exe")
drivers.get(site)

#Clicking Funding details options
try:
    WebDriverWait(drivers, 5).until(EC.element_to_be_clickable((By.XPATH, '//li[@class="bs-pretty-tabs-container menu-item-has-children menu-item better-anim-fade bsm-leave"]'))).click()
    WebDriverWait(drivers, 10).until(EC.element_to_be_clickable((By.XPATH, '//li[@id="menu-item-51510"]'))).click()
except:
    WebDriverWait(drivers, 20).until(EC.element_to_be_clickable((By.XPATH, '//li[@id="menu-item-51510"]'))).click()
    
#Clicking display options
WebDriverWait(drivers, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tablepress-48_length"]/label/select/option[3]'))).click()
#Clicking display options
WebDriverWait(drivers, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tablepress-49_length"]/label/select/option[3]'))).click()
#Clicking display options
WebDriverWait(drivers, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tablepress-50_length"]/label/select/option[3]'))).click()

#getting the page source 
time.sleep(2)
html = drivers.page_source
#Reading the tabels in HTML
df = pd.read_html(html)
#Storing the monthly data
sept = df[15]
aug = df[17]
july = df[19]
#Concating the whole data 
df_row_reindex = pd.concat([sept, aug, july], ignore_index=True)
df_row_reindex.drop("Sr. No.",axis='columns', inplace=True)
df_row_reindex

,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD)
0,08/09/2020,Byju’s,EduTech,Online Tutoring,Bangalore,"Silver Lake, Tiger Global, General Atlantic an...",Private Equity,500000000
1,12/09/2020,mCaffeine,Personal Care,Skincare & Haircare,Mumbai,"Amicus Capital Private Equity I LLP, Amicus Ca...",Series B,3000000
2,09/09/2020,Qshala,EduTech,Online Curiosity Platform for Kids,Bangalore,Rainmatter Capital,Angel,370000
3,02/09/2020,Winzo,Online Gaming,Online Gaming,New Delhi,"Kalaari Capital Partners, IndigoEdge Managemen...",Series B,15500000
4,09/09/2020,Hippo Video,Video Customer Experience(CX) Platform,Video Customer Experience(CX) Platform,"Newark, Delaware, United States of Amercia","Alpha Wave Incubation, Exfinity Venture Partne...",Series A,4500000
5,07/09/2020,Melorra,E-commerce,Online Jewelry Store,Bangalore,"Shadow Holdings, Lightbox.",Debt Financing,"upto 8,900,000"
6,07/09/2020,1mg,E-commerce,Online Pharmacy,Gurgaon,"Gaja Capital, Tata Capital, Partners Group",In Progress,100000000
7,31/08/2020,mfine,HealthTech,On-Demand Healthcare Services,Bangalore,Caretech Pte Inc,Series B,5400000
8,31/08/2020,Apna,Human Resources,Recruitment Platform,Bangalore,Lightspeed India and Sequoia Capital India,Series A,8000000
9,03/09/2020,Railofy,Transportation,WL & RAC protection platform,Mumbai,Chiratae Ventures,Seed,950000


# Q7.Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [ ]:
site = 'https://www.digit.in/'

In [337]:
drivers = webdriver.Chrome(r"C://chromedriver.exe")
drivers.get('https://www.digit.in/')
WebDriverWait(drivers, 5).until(EC.element_to_be_clickable((By.XPATH, '//div[@class="menu"]/ul/li[4]/a'))).click()
filter_btn=drivers.find_element_by_xpath('//button[@class="tablinks "][1]')
filter_btn.click()
cat_btn=drivers.find_element_by_xpath('//div[@id="laptops"]/div[3]')
cat_btn.click()
html = drivers.page_source
df = pd.read_html(html)

In [345]:
main= df[0]

In [420]:
main

,Product Name,Seller,Price
0,MSI GT76 TITAN DT 9SG,NaN,₹379990
1,Alienware 17 Area-51m,amazon,₹422000
2,HP Omen 15 2020,Tatacliq,₹117790
3,Asus Zephyrus G14,amazon,₹164990
4,Lenovo Legion Y540,flipkart,₹79990
5,Asus ROG Zephyrus G GA502,flipkart,₹79990
6,Asus ROG Zephyrus S GX531,flipkart,₹239990
7,MSI GT83VR 7RE Titan SLI,NaN,₹349990
8,ASUS ROG ZEPHYRUS DUO 15,amazon,₹244990
9,Dell G3,amazon,₹61990


In [418]:
#specifications of Laptop from 1-10
lapt1 = df[1]
lapt1.drop('SPECIFICATION.1',axis='columns',inplace= True)
lapt1 = lapt1.T
headers = lapt1.iloc[0]
lapt1  = pd.DataFrame(lapt1.values[1:], columns=headers)

lapt2 = df[2]
lapt2.drop('SPECIFICATION.1',axis='columns',inplace= True)
lapt2 = lapt2.T
headers = lapt2.iloc[0]
lapt2  = pd.DataFrame(lapt2.values[1:], columns=headers)

lapt3 = df[4]
lapt3.drop('SPECIFICATION.1',axis='columns',inplace= True)
lapt3 = lapt3.T
headers = lapt3.iloc[0]
lapt3  = pd.DataFrame(lapt3.values[1:], columns=headers)

lapt4 = df[6]
lapt4.drop('SPECIFICATION.1',axis='columns',inplace= True)
lapt4 = lapt4.T
headers = lapt4.iloc[0]
lapt4  = pd.DataFrame(lapt4.values[1:], columns=headers)

lapt5 = df[8]
lapt5.drop('SPECIFICATION.1',axis='columns',inplace= True)
lapt5 = lapt5.T
headers = lapt5.iloc[0]
lapt5  = pd.DataFrame(lapt5.values[1:], columns=headers)

lapt6 = df[10]
lapt6.drop('SPECIFICATION.1',axis='columns',inplace= True)
lapt6 = lapt6.T
headers = lapt6.iloc[0]
lapt6  = pd.DataFrame(lapt6.values[1:], columns=headers)

lapt7 = df[12]
lapt7.drop('SPECIFICATION.1',axis='columns',inplace= True)
lapt7 = lapt7.T
headers = lapt7.iloc[0]
lapt7  = pd.DataFrame(lapt7.values[1:], columns=headers)

lapt8 = df[14]
lapt8.drop('SPECIFICATION.1',axis='columns',inplace= True)
lapt8 = lapt8.T
headers = lapt8.iloc[0]
lapt8  = pd.DataFrame(lapt8.values[1:], columns=headers)

lapt9 = df[15]
lapt9.drop('SPECIFICATION.1',axis='columns',inplace= True)
lapt9 = lapt9.T
headers = lapt9.iloc[0]
lapt9  = pd.DataFrame(lapt9.values[1:], columns=headers)

lapt10 = df[17]
lapt10.drop('SPECIFICATION.1',axis='columns',inplace= True)
lapt10 = lapt10.T
headers = lapt10.iloc[0]
lapt10  = pd.DataFrame(lapt10.values[1:], columns=headers)

df1 = pd.concat([lapt1,lapt2,lapt3,lapt4,lapt5,lapt6,lapt7,lapt8,lapt9,lapt10], ignore_index=True)

In [419]:
df1

,OS,Display,Processor,Memory,Weight,Dimension,Graphics Processor,Price
0,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,₹379990
1,Windows 10 Pro,"17.3"" (FHD (1920 x 1080))",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB PCIe SSD/32GB DDR4,3.87 Kg,42 mm x 402.6 mm x 319.14 mm,NVIDIA GeForce RTX 2080,NaN
2,Windows 10 Home,"15.6"" (1920 x 1080)",Intel i7-10750H 10th Gen | 1.6GHz,512 GB SSD/16 GBGB DDR4,5.40,14.09 x 9.44 x 0.89,Nvidia GeForce GTX 1650Ti,NaN
3,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Generation Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,NaN
4,Windows 10 Home,"15.6"" (1920 X 1080)",9th Generation Core Intel I7-9750H | 2.6 Ghz,1 TB SSD/8GB DDR4,2.3,365mm x 260mm x 25.9mm,NVIDIA® GeForce RTX™ 2060,NaN
5,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen 7 Quad Core 3750H | 2.3 GHz,512 GB SSD/16GB DDR4,2.2,360 x 252 x 20.4,NVIDIA Geforce GTX 1660 Ti,NaN
6,Windows 10 Home,"15.6"" (1920 X 1080)",9th Gen Intel Core i7-8750H | 2.2 GHz,512GB SSD/16 GBGB DDR4,2.1,360 (W) x 268 (D) x 15.35~16.15 (H) mm,NVIDIA® GeForce RTX™ 2070 (Max-Q),NaN
7,Windows 10 Home 64 bit,"18.4"" (1920 x 1080)",Intel CM238 Core i7-7820HK+CM238 7th Gen | 3.5GHz,1.5 TB SATA/64GB DDR4,5.5,458 x 339 x 69,Dual GTX1070,₹349990
8,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,NaN
9,Windows 10 Home Plus,15.6 MP | NA,8th Gen Intel core i5-8300H | 2.3GHz,1TB HDD/8GB DDR4,2.53,22.7 X 380 X 258,NVidia GeForce GTX 1050,NaN


In [424]:
df1.drop('Price',axis='columns',inplace= True)

In [433]:
df_final = pd.concat([main, df1], axis=1, ignore_index=True)

In [437]:
df_final.columns=['Product Name','Seller', 'OS','Price','Display','Processor','Memory','Weight','Dimension','Graphics Processor']

In [438]:
df_final

,Product Name,Seller,OS,Price,Display,Processor,Memory,Weight,Dimension,Graphics Processor
0,MSI GT76 TITAN DT 9SG,NaN,₹379990,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080
1,Alienware 17 Area-51m,amazon,₹422000,Windows 10 Pro,"17.3"" (FHD (1920 x 1080))",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB PCIe SSD/32GB DDR4,3.87 Kg,42 mm x 402.6 mm x 319.14 mm,NVIDIA GeForce RTX 2080
2,HP Omen 15 2020,Tatacliq,₹117790,Windows 10 Home,"15.6"" (1920 x 1080)",Intel i7-10750H 10th Gen | 1.6GHz,512 GB SSD/16 GBGB DDR4,5.40,14.09 x 9.44 x 0.89,Nvidia GeForce GTX 1650Ti
3,Asus Zephyrus G14,amazon,₹164990,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Generation Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060
4,Lenovo Legion Y540,flipkart,₹79990,Windows 10 Home,"15.6"" (1920 X 1080)",9th Generation Core Intel I7-9750H | 2.6 Ghz,1 TB SSD/8GB DDR4,2.3,365mm x 260mm x 25.9mm,NVIDIA® GeForce RTX™ 2060
5,Asus ROG Zephyrus G GA502,flipkart,₹79990,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen 7 Quad Core 3750H | 2.3 GHz,512 GB SSD/16GB DDR4,2.2,360 x 252 x 20.4,NVIDIA Geforce GTX 1660 Ti
6,Asus ROG Zephyrus S GX531,flipkart,₹239990,Windows 10 Home,"15.6"" (1920 X 1080)",9th Gen Intel Core i7-8750H | 2.2 GHz,512GB SSD/16 GBGB DDR4,2.1,360 (W) x 268 (D) x 15.35~16.15 (H) mm,NVIDIA® GeForce RTX™ 2070 (Max-Q)
7,MSI GT83VR 7RE Titan SLI,NaN,₹349990,Windows 10 Home 64 bit,"18.4"" (1920 x 1080)",Intel CM238 Core i7-7820HK+CM238 7th Gen | 3.5GHz,1.5 TB SATA/64GB DDR4,5.5,458 x 339 x 69,Dual GTX1070
8,ASUS ROG ZEPHYRUS DUO 15,amazon,₹244990,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q
9,Dell G3,amazon,₹61990,Windows 10 Home Plus,15.6 MP | NA,8th Gen Intel core i5-8300H | 2.3GHz,1TB HDD/8GB DDR4,2.53,22.7 X 380 X 258,NVidia GeForce GTX 1050
